# Imports

In [54]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch_geometric.data import Data
from torch_geometric.nn import GraphConv
from torch_geometric.data import DataLoader


file_path = "water-DZ-F12-STATIC-g32n10-3M_PES-DZERO.xyz"

## Parse data

In [24]:
def remove_between_asterisks(line):
    start_index = line.find('*')
    end_index = line.rfind('*')
    if start_index != -1 and end_index != -1:
        return line[:start_index] + line[end_index + 1:]
    else:
        return line

with open(file_path, 'r') as file:
    lines = file.readlines()

# Process each line and remove characters between '*' symbols
for i in range(len(lines)):
    lines[i] = remove_between_asterisks(lines[i])

# Write the modified content back to the file
with open(file_path, 'w') as file:
    file.writelines(lines)


### Create data objects from dataset for using PyTorch Geometric

In [149]:
from torch_geometric.data import Data

def process_dataset(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()

    # Initialize lists to store node features, edge indices, and target values
    node_features = []
    edge_indices = []
    targets = []

    # Process each molecule block in the dataset
    i = 0
    while i < len(lines):
        # Skip lines until we find a block with atomic coordinates
        while i < len(lines) and not lines[i].startswith('O'):
            i += 1

        # Extract energy value
        energy_line = lines[i-1]  # Use the line before the atomic coordinates

        # Split the line using '=' as a separator
        energy_parts = energy_line.split('=')

        # Check if there are enough parts
        if len(energy_parts) > 1:
            # Extract energy value
            energy = float(energy_parts[1].strip())
            targets.append(energy)
        else:
            print("Error: Could not extract energy value from the line.")

        # Process atom coordinates
        i += 1
        node_feats = []
        while i < len(lines) and not lines[i].startswith('3'):
            parts = lines[i].split()
            atom_type = parts[0]
            coordinates = [float(parts[j]) for j in range(1, 4)]
            node_feats.append(coordinates)
            i += 1

        # Store node features
        node_features.append(node_feats)

        # Generate edge indices based on connectivity (assuming a fully connected graph)
        num_atoms = len(node_feats)
        edge_index = torch.tensor([[i, j] for i in range(num_atoms) for j in range(i+1, num_atoms)], dtype=torch.long).t().contiguous()
        edge_indices.append(edge_index)

    # Convert to PyTorch Geometric Data objects
    data_list = []
    for nodes, edges, target in zip(node_features, edge_indices, targets):
        x = torch.tensor(nodes, dtype=torch.float)
        edge_index = edges.view(2, -1).long()  # Ensure the correct type
        y = torch.tensor([target], dtype=torch.float)
        data = Data(x=x, edge_index=edge_index, y=y)
        data_list.append(data)

    return data_list


In [150]:
dataset = process_dataset(file_path)

In [151]:
print(dataset)

[Data(x=[2, 3], edge_index=[2, 1], y=[1]), Data(x=[2, 3], edge_index=[2, 1], y=[1]), Data(x=[2, 3], edge_index=[2, 1], y=[1]), Data(x=[2, 3], edge_index=[2, 1], y=[1]), Data(x=[2, 3], edge_index=[2, 1], y=[1]), Data(x=[2, 3], edge_index=[2, 1], y=[1]), Data(x=[2, 3], edge_index=[2, 1], y=[1]), Data(x=[2, 3], edge_index=[2, 1], y=[1]), Data(x=[2, 3], edge_index=[2, 1], y=[1]), Data(x=[2, 3], edge_index=[2, 1], y=[1]), Data(x=[2, 3], edge_index=[2, 1], y=[1]), Data(x=[2, 3], edge_index=[2, 1], y=[1]), Data(x=[2, 3], edge_index=[2, 1], y=[1]), Data(x=[2, 3], edge_index=[2, 1], y=[1]), Data(x=[2, 3], edge_index=[2, 1], y=[1]), Data(x=[2, 3], edge_index=[2, 1], y=[1]), Data(x=[2, 3], edge_index=[2, 1], y=[1]), Data(x=[2, 3], edge_index=[2, 1], y=[1]), Data(x=[2, 3], edge_index=[2, 1], y=[1]), Data(x=[2, 3], edge_index=[2, 1], y=[1]), Data(x=[2, 3], edge_index=[2, 1], y=[1]), Data(x=[2, 3], edge_index=[2, 1], y=[1]), Data(x=[2, 3], edge_index=[2, 1], y=[1]), Data(x=[2, 3], edge_index=[2, 1],

## Definition of GNN Class and functionalities


In [152]:
class MyGNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(MyGNN, self).__init__()
        self.conv1 = GraphConv(input_dim, hidden_dim)
        self.conv2 = GraphConv(hidden_dim, output_dim)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = torch.relu(x)
        x = self.conv2(x, edge_index)
        return x

In [153]:
# Split the dataset into training and testing sets
train_dataset = dataset[:2]
test_dataset = dataset[2:]

In [154]:
# Create DataLoader for batch processing
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

c:\Python\Python3.9.2\lib\site-packages\torch_geometric\deprecation.py:22: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [155]:
# Define the model, loss function, and optimizer
model = MyGNN(input_dim=3, hidden_dim=64, output_dim=1)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [156]:
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    for data in train_loader:
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, data.y.view(-1, 1))  # Assuming data.y contains energy values
        loss.backward()
        optimizer.step()

c:\Python\Python3.9.2\lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([1, 1])) that is different to the input size (torch.Size([2, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


In [158]:
import numpy as np
from sklearn.metrics import mean_squared_error

# Initialize empty arrays
all_true_energies = []
all_predicted_energies = []
num_predictions_per_input = []

# Evaluation
model.eval()
with torch.no_grad():
   for data in test_loader:
    output = model(data)
    
    # Assuming output is a tensor with shape [batch_size, ...]
    if len(output.shape) == 2:
        predicted_energies = output.squeeze().tolist()
    elif len(output.shape) == 1:
        predicted_energies = [output.item()]
    else:
        raise ValueError("Unexpected shape of model output.")

    true_energies = data.y.squeeze().tolist()

    # Ensure true_energies is iterable
    if not isinstance(true_energies, (list, tuple)):
        true_energies = [true_energies]

    # Append to arrays
    all_true_energies.extend(true_energies)
    all_predicted_energies.extend(predicted_energies)
    
    print(true_energies, predicted_energies)
    
""" # Convert to NumPy arrays
all_true_energies = np.array(all_true_energies)
all_predicted_energies = np.array(all_predicted_energies)

# Calculate RMSE
rmse = np.sqrt(mean_squared_error(all_true_energies, all_predicted_energies))

print(f"RMSE: {rmse}") """


[-76.25068664550781] [-0.5705940127372742, -1.1459054946899414]
[-76.24559020996094] [-0.7483702898025513, -1.3471180200576782]
[-76.26419830322266] [-0.705357015132904, -1.3012077808380127]
[-76.27043151855469] [-0.5707038640975952, -1.1272883415222168]
[-76.2527847290039] [-0.5769790410995483, -1.1522552967071533]
[-76.26285552978516] [-0.7116169929504395, -1.3099095821380615]
[-76.26802825927734] [-0.5834864377975464, -1.1466262340545654]
[-76.26697540283203] [-0.6183186173439026, -1.1871296167373657]
[-76.24417114257812] [-0.5461946725845337, -1.1219778060913086]
[-76.2675552368164] [-0.6255309581756592, -1.1954212188720703]
[-75.92100524902344] [-0.43699565529823303, -0.8828194737434387]
[-75.88088989257812] [-0.4259330928325653, -0.8691067695617676]
[-75.95331573486328] [-0.44844895601272583, -0.8961947560310364]
[-75.85794830322266] [-0.4214528203010559, -0.8613294363021851]
[-75.98788452148438] [-0.4655246138572693, -0.9164133667945862]
[-76.00270080566406] [-0.4790146946907043

KeyboardInterrupt: 

In [145]:
import torch
import numpy as np
from sklearn.metrics import r2_score

# Assuming you have the true and predicted energies as NumPy arrays
true_energies = np.array(all_true_energies)  # Replace with your actual true energies
predicted_energies = np.array(all_predicted_energies)  # Replace with your actual predicted energies

# Check lengths
if len(true_energies) != len(predicted_energies):
    raise ValueError("Lengths of true and predicted energies are inconsistent.")

# Calculate R-squared score
r2 = r2_score(true_energies, predicted_energies)

print(f"R-squared Score: {r2}")


ValueError: Lengths of true and predicted energies are inconsistent.